In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
143,2024-08-09,República Dominicana Lnb,21:00,Indios,Titanes Del Licey,1.52,2.29,174.5,1.83,1.83,0.0,0.00,0.00,hANprqe3,0.657895,0.436681,0.546448,0.546448,0.094576,137.084,13.320054,0.097167,1.2,1.643168,1.369306,145.52,1.648,0.301281,0.182816,-4.0,166.008,45.093027,0.271632,1.8,1.643168,0.912871,116.20,1.950,0.623458,0.319722,17.0,107,70,1.36,1.66,0.0,0.0,0.285812,0.000000,NaN,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
144,2024-08-09,República Dominicana Lnb,20:30,Marineros,Reales de la Vega,2.29,1.52,185.5,1.80,1.86,0.0,0.00,0.00,CEpbKu3d,0.436681,0.657895,0.555556,0.537634,0.094576,167.782,40.912303,0.243842,0.6,1.341641,2.236068,238.85,2.080,0.462979,0.222586,-49.0,129.982,23.991045,0.184572,2.4,1.341641,0.559017,146.20,1.528,0.233174,0.152601,26.0,85,85,2.81,1.72,0.0,0.0,0.285812,0.023184,NaN,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
145,2024-08-09,República Dominicana Lnb,21:30,Metros de Santiago,Caneros,1.35,2.86,178.5,1.85,1.81,0.0,0.00,0.00,0zRhtNPF,0.740741,0.349650,0.540541,0.552486,0.090391,222.280,59.319098,0.266867,1.2,1.643168,1.369306,183.60,2.410,0.664304,0.275645,-1.0,148.700,26.975253,0.181407,3.0,0.000000,0.000000,148.19,1.722,0.311721,0.181023,22.0,85,73,2.16,2.03,0.0,0.0,0.507236,0.015456,NaN,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
146,2024-08-09,Mundo Jogos Olímpicos,16:00,França F,Bélgica F,1.53,2.59,143.5,1.88,1.90,-4.5,1.87,3.10,dCVHfUY1,0.653595,0.386100,0.531915,0.526316,0.039695,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,84,79,1.40,1.41,0.0,0.0,0.363851,0.007483,0.349997,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
147,2024-08-09,Mundo Jogos Olímpicos,12:30,EUA F,Austrália F,1.06,10.50,161.5,1.86,1.92,0.0,0.00,0.00,tp1eGFsH,0.943396,0.095238,0.537634,0.520833,0.038634,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,101,85,1.01,1.28,0.0,0.0,1.154860,0.022448,NaN,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
148,2024-08-09,México Lnbp,00:00,Soles,Diablos Rojos,3.02,1.35,161.5,1.86,1.84,6.5,1.83,1.24,fg8AtCrI,0.331126,0.740741,0.537634,0.543478,0.071867,224.536,125.172609,0.557472,1.2,1.643168,1.369306,226.00,2.608,1.628594,0.624461,14.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,100,78,2.26,1.59,0.0,0.0,0.540443,0.007644,0.271787,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
149,2024-08-09,Vietnã Vba,09:30,Hanoi Buffaloes,Ho Chi Minh City Wings,1.35,2.86,164.5,1.83,1.83,0.0,0.00,0.00,rgNXaeuq,0.740741,0.349650,0.546448,0.546448,0.090391,170.852,62.375299,0.365084,1.8,1.643168,0.912871,217.30,2.072,0.719944,0.347464,-16.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,82,78,2.65,2.56,0.0,0.0,0.507236,0.000000,NaN,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
